In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [2]:
classes = ['Alzheimer', 'MCI' , 'Normal']

data = {}
labels = {}
for class_ in classes:
    dirs_ = os.listdir(class_)
    data_ = []
    for dir_ in dirs_:
        files_path =os.path.join(os.path.join(class_, dir_), '*.nii')
        for file_ in glob.glob(files_path):
            temp_file = nib.load(file_).get_data()
            if temp_file.shape[0] != 64 or temp_file.shape[1] != 64 or temp_file.shape[2] !=48:
                continue
            data_.append(temp_file)
    data[class_] = np.array(data_)

working on Alzheimer...
working on MCI...
working on Normal...


In [3]:
temp_concat = np.concatenate((data['Alzheimer'][0], data['Alzheimer'][1]), axis=3)
for i in range(2, data['Alzheimer'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Alzheimer'][i]), axis=3)
data['Alzheimer'] = temp_concat
print data['Alzheimer'].shape

(64, 64, 48, 6020)


In [4]:
# temp_concat = np.concatenate((data['Alzheimer _ 70'][0], data['Alzheimer _ 70'][1]), axis=3)
# for i in range(2, data['Alzheimer _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Alzheimer _ 70'][i]), axis=3)
# data['Alzheimer _ 70'] = temp_concat
# print data['Alzheimer _ 70'].shape

In [5]:
temp_concat = np.concatenate((data['MCI'][0], data['MCI'][1]), axis=3)
for i in range(2, data['MCI'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['MCI'][i]), axis=3)
data['MCI'] = temp_concat
print data['MCI'].shape

(64, 64, 48, 5320)


In [6]:
# temp_concat = np.concatenate((data['MCI _ 70'][0], data['MCI _ 70'][1]), axis=3)
# for i in range(2, data['MCI _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['MCI _ 70'][i]), axis=3)
# data['MCI _ 70'] = temp_concat
# print data['MCI _ 70'].shape

In [7]:
temp_concat = np.concatenate((data['Normal'][0], data['Normal'][1]), axis=3)
for i in range(2, data['Normal'].shape[0]):
    temp_concat = np.concatenate((temp_concat, data['Normal'][i]), axis=3)
data['Normal'] = temp_concat
print data['Normal'].shape

(64, 64, 48, 5880)


In [8]:
# temp_concat = np.concatenate((data['Normal _ 70'][0], data['Normal _ 70'][1]), axis=3)
# for i in range(2, data['Normal _ 70'].shape[0]):
#     temp_concat = np.concatenate((temp_concat, data['Normal _ 70'][i]), axis=3)
# data['Normal _ 70'] = temp_concat
# print data['Normal _ 70'].shape

In [9]:
np.save('Normal',data['Normal'])

In [10]:
np.save('Alzheimer',data['Alzheimer'])

In [11]:
np.save('MCI',data['MCI'])

## Checkpoint


In [84]:
normal=np.load('Normal.npy')

In [3]:
mci=np.load('MCI.npy')

In [4]:
alzheimer=np.load('Alzheimer.npy')

In [5]:
alzheimer.shape

(64, 64, 48, 6020)

In [6]:
mci.shape

(64, 64, 48, 5320)

In [7]:
print mci.shape

(64, 64, 48, 2380)

In [8]:
combined_data=[]
for i in range(alzheimer.shape[3]):
    combined_data.append((alzheimer[:,:,:,i],0))
for i in range(normal.shape[3]):
    combined_data.append((normal[:,:,:,i],2))
for i in range(mci.shape[3]):
    combined_data.append((mci[:,:,:,i],1))
    
del(alzheimer)
del(normal)
del(mci)

In [9]:
np.array(combined_data).shape

(17220, 2)

In [10]:
# shuffling of combined_data
random.shuffle(combined_data)
print len(combined_data)

17220


In [11]:
# get input and output.
X = map(lambda x: x[0],combined_data)
print np.array(X).shape
y = map(lambda x:x[1], combined_data)
print len(y)

(17220, 64, 64, 48)
17220


## Neural Network

###  Prepare Data

In [12]:
x_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.4)

In [13]:
np.array(y_val).shape

(6888,)

In [14]:
short_x_train=x_train[:5000]
short_y_train=y_train[:5000]

In [15]:
short_x_val=x_val[:1000]
short_y_val=y_val[:1000]

In [16]:
def batch_generator(X, y, batch_size,samples_per_epoch):
    number_of_batches = (samples_per_epoch)/batch_size
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index,:]
    y =  y[shuffle_index]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch,:]
        y_batch = y[index_batch]
        counter += 1
        yield(np.array(X_batch),y_batch)
        if (counter < number_of_batches):
            np.random.shuffle(shuffle_index)
            counter=0

In [17]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
 
# initialize the model
model = Sequential()

#first layer
model.add(Convolution2D(96, 5, 5, border_mode="same",
    input_shape=(64, 64, 48)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#second layer
model.add(Convolution2D(144, 5, 5, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set of FC
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))

#get model summary
model.summary()

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 96)        115296    
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 144)       345744    
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 144)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 144)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
__________

/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (5, 5), padding="same", input_shape=(64, 64, 4...)`
/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(144, (5, 5), padding="same")`


In [18]:
model.fit(np.asarray(short_x_train), np.asarray(short_y_train),nb_epoch=5,batch_size=250,
          validation_data=(np.asarray(short_x_val),np.asarray(short_y_val)))
                    

/home/siddhant/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 5000 samples, validate on 1000 samples
Epoch 1/5
5000/5000 [==============================] - 434s - loss: 11.1955 - acc: 0.3044 - val_loss: 10.7508 - val_acc: 0.3330
Epoch 2/5
5000/5000 [==============================] - 428s - loss: 11.1892 - acc: 0.3058 - val_loss: 10.7508 - val_acc: 0.3330
Epoch 3/5
5000/5000 [==============================] - 522s - loss: 11.1892 - acc: 0.3058 - val_loss: 10.7508 - val_acc: 0.3330
Epoch 4/5
5000/5000 [==============================] - 560s - loss: 11.1892 - acc: 0.3058 - val_loss: 10.7508 - val_acc: 0.3330
Epoch 5/5
5000/5000 [==============================] - 419s - loss: 11.1892 - acc: 0.3058 - val_loss: 10.7508 - val_acc: 0.3330


In [19]:
model.save_weights('Le-Net_model_weights') 